In [1]:
import geopandas
import pandas as pd
import glob
import shapefile
from tqdm import tqdm
import numpy as np
import pickle
import xlrd3
import os
import glob
import openpyxl

In [12]:
# read poi and locations from shp files
full_names = pd.read_excel('/home/yangwenhao/local/project/weibo_filter/app/data/POI_tianjin_0427.xlsx',
                              sheet_name=0)

# 经度	纬度
# names2loc = {}
# for l in places_names_loc.iterrows():
#     # print(l[1].values[0], l[1].values[1],)
#     names2loc[l[1].values[0]] = l[1].values[1]

# with open('names2loc.pickle', 'wb') as f:
#     pickle.dump(names2loc, f)

In [37]:
class location(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [39]:
names2loc = {}
for x in full_names[['名称', '经度', '纬度']].iterrows():
    # print(x)
    loc = location(x=x[1].经度, y=x[1].纬度)
    names2loc[x[1].名称] = loc
    
with open('names2loc2.pickle', 'wb') as f:
    pickle.dump(names2loc, f)

In [10]:
# create place2index files giving tweets and locations
# pickle file: [places, sparse_matirx]

sub_names = ['商业娱乐', '文化展览', '科研教育', '体育健身', '住区', '旅游景点', '企业办公', '公共服务', '交通设施', '医养设施']

for sht_idx in range(10, 11):
    full_names = pd.read_excel('/home/yangwenhao/local/project/weibo_filter/app/data/POI_tianjin_0427.xlsx',
                              sheet_name=sht_idx)
    names = full_names.名称
    location_names = names
    numofprint = 4

    for lst in ['list2_img', 'list2_woimg']:
        text_csv = 'all_list/' + '%s.csv'%(lst)
        if not os.path.exists(text_csv):
            data = xlrd3.open_workbook('/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xlsx')
            table = data.sheet_by_index(0)#.sheets()[0]          #通过索引顺序获取
            # get full-text tweets
            texts = []
            if lst == 'first_2w':
                test_idx = 5
            elif lst == 'list2_img':
                test_idx = 4
            elif lst == 'list2_woimg':
                test_idx = 6
            else:
                test_idx = 6

            for i in range(1, table.nrows):
                texts.append(table.row(i)[test_idx].value)

            texts = pd.DataFrame(texts, columns=['text'])
            texts.to_csv('%s.csv'%(lst), index=False)
        else:
            texts = pd.read_csv(text_csv)
        # match all palces for all lines
        places,sparse_matirx = [],[]
        for idx, i in tqdm(enumerate(location_names.unique()), ncols=50,
                           total=len(location_names.unique())): #[297604:]
            try:
                result = texts.text.str.contains(i) # 12:26:26
                if result.to_numpy().sum() > 0:
                    places.append(i)
                    sparse_matirx.append(result.to_numpy())
            except Exception as e:
                continue

            if (idx+1) % 250000 == 0 or (idx+1) == len(location_names.unique()):
                print("出现的地点数: ", len(places), " 记录的列数:", len(sparse_matirx))
                with open('all_list2/%dplace2index_%s_%d.pickle' % (sht_idx, lst, idx), 'wb') as f:
                    pickle.dump([places, sparse_matirx], f)
                places,sparse_matirx = [],[]

100%|█| 10028/10028 


出现的地点数:  265  记录的列数: 265


100%|█| 10028/10028 

出现的地点数:  0  记录的列数: 0


In [ ]:
from multiprocessing import Pool

pool = Pool(processes=14)  # 创建nj个进程
loc_csv = 'tianjin.csv'

for tweet_csv in text_csvs:
    # MatchProcess(tweet_csv, loc_csv, pickle_path)
    pickle_path = tweet_csv.replace('csv', 'pickle')
    pool.apply_async(MatchProcess, args=(tweet_csv, loc_csv, pickle_path))
    
pool.close()  # 关闭进程池，表示不能在往进程池中添加进程
pool.join()  # 等待进程池中的所有进程执行完毕，必须在close()之后调用

In [ ]:
# create xlsx files giving pickle file: [places, sparse_matirx]
sub_names = ['商业娱乐', '文化展览', '科研教育', '体育健身', '住区', '旅游景点', '企业办公', '公共服务', '交通设施', '医养设施']

for sht_idx in range(1, 11):
    numofprint = 4
    for lst in ['list2_img', 'list2_woimg']:
        text_csv = 'all_list/' + '%s.csv'%(lst)
        if not os.path.exists(text_csv):
            data = xlrd3.open_workbook('/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xlsx')
            table = data.sheet_by_index(0)#.sheets()[0]          #通过索引顺序获取
            # get full-text tweets
            texts = []
            if lst == 'first_2w':
                test_idx = 5
            elif lst == 'list2_img':
                test_idx = 4
            elif lst == 'list2_woimg':
                test_idx = 6
            else:
                test_idx = 6

            for i in range(1, table.nrows):
                texts.append(table.row(i)[test_idx].value)

            texts = pd.DataFrame(texts, columns=['text'])
            texts.to_csv('%s.csv'%(lst), index=False)
        else:
            texts = pd.read_csv(text_csv)
        # match all palces for all lines
        places,sparse_matirx = [],[]
        for idx, i in tqdm(enumerate(location_names.unique()), ncols=50,
                           total=len(location_names.unique())): #[297604:]
            try:
                result = texts.text.str.contains(i) # 12:26:26
                if result.to_numpy().sum() > 0:
                    places.append(i)
                    sparse_matirx.append(result.to_numpy())
            except Exception as e:
                continue

            if (idx+1) % 250000 == 0 or (idx+1) == len(location_names.unique()):
                print("出现的地点数: ", len(places), " 记录的列数:", len(sparse_matirx))
                with open('all_list2/%dplace2index_%s_%d.pickle' % (sht_idx, lst, idx), 'wb') as f:
                    pickle.dump([places, sparse_matirx], f)
                places,sparse_matirx = [],[]

In [65]:
invalid_places = set(['内', '顾', '尚', 'R', '简', '锦', '落', '嚟', 'C', 'e', 'f', 'H', 'L', 'n', 'W', '酷', '漫', '品', '陶', '喜', '懿', '银', '渔', '遇', '青', '佛', '中', 'A+', '北京', '入胜', '佛', '英语', '乐高', '球球', '大地', '完美'])

In [ ]:
# save match results with pickle
headers = ["相关地点数量", "微博id", "微博主页", "文本", "日期", "图片", "转发数", "评论数", "点赞数"]
names2loc_key = set(list(names2loc.keys()))

for lst in ['list2_img', 'list2_woimg']:
    
    text_csv = 'all_list/' + '%s.csv'%(lst)
    text_pickle = 'all_list2/' + '*_%s_*.pickle'%(lst)
    text_pickles = glob.glob(text_pickle)
    this_texts = pd.read_csv(text_csv, index_col=0)
    
    for t in text_pickles:
        append_cells = {}
        with open(t, 'rb') as f:
            [places, sparse_matirx] = pickle.load(f)
        places_np = np.array(places)
        sparse_np = np.array(sparse_matirx) #.shape
        if places_np.shape[0] > 0:
            for j, l in zip(this_texts.index, range(sparse_np.shape[1])):
                row_cells = []
                for i in places_np[np.where(sparse_np[:, l] == True)[0]]:
                    if i in invalid_places: # 过滤奇怪的点
                        continue
                        
                    if i in names2loc_key:
                        i_xy = names2loc[i]
                        row_cells.append([i, i_xy.x, i_xy.y])
                    else:
                        row_cells.append([i, "", ""])
                append_cells[j] = row_cells
        else:
            print('Skip %s ...'%(t))
            continue
            
        if lst == 'list2_img':
            read_cols = [2, 3, 4, 5, 6, 7, 8, 9]
        elif lst == 'list2_woimg':
            read_cols = [2, 3, 6, 7, 8, 9, 10, 11]

        read_lst = r'/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xlsx'
        if t.split('/')[-1][1].isdigit():
            title_name = sub_names[int(t.split('/')[-1][:2])-1]
        else:
            title_name = sub_names[int(t.split('/')[-1][0])-1]
            #title_name = sub_names[int(t.split('/')[-1][0])-1]
        
        save_lst = 'all_list2/' + title_name + '.xlsx' # t.replace('pickle', 'loc) 
        read_writebook = xlrd3.open_workbook(filename=read_lst)
        writebook = openpyxl.Workbook()
        # 添加sheet 写入excel, 参数对应 行, 列, 值
        # print(title_name, "...", end=' ')
        sheet      = writebook.create_sheet(title=title_name, index=0)
        sht_row = 2 # openpyxl start from 1 instead of 0
        # 相关地点数量	微博id	微博主页	文本	日期	图片	转发数	评论数	点赞数
        for i, h in enumerate(headers):
            sheet.cell(1, i+1, h)

        # write header
        # find the tweet with the largest number of locations
        max_los = 1
        for i in append_cells:
            max_los = max(max_los, len(append_cells[i]))
            
        for i in range(1, max_los+1):
            sheet.cell(1, (i-1)*3+len(headers)+1, '地点提取%d'%i)
            sheet.cell(1, (i-1)*3+len(headers)+2, '经度%d'%i)
            sheet.cell(1, (i-1)*3+len(headers)+3, '纬度%d'%i)

        read_table = read_writebook.sheet_by_index(0)
        nrows = read_table.nrows
        assert (nrows-1) == len(append_cells), print("rows in excel: ", (nrows-1), "rows in loc_csv: ", len(append_cells))
        
        for i in tqdm(range(1, nrows), ncols=100):
            this_line = read_table.row_slice(i)
            this_locs = append_cells[i-1]
            locs_names = [l[0] for l in this_locs]

            if len(this_locs) > 0:
                this_row = sht_row
                sht_row += 1

                sheet.cell(this_row, 1, len(this_locs))     
                for j, c in enumerate(read_cols):
                    sheet.cell(this_row, j+2, this_line[c].value)

                for j, (name, x, y) in enumerate(this_locs):
                    sheet.cell(this_row, j*3+len(headers)+1,   name)
                    sheet.cell(this_row, j*3+len(headers)+2,   x)
                    sheet.cell(this_row, j*3+len(headers)+3,   y)

        writebook.save(save_lst)
        print(save_lst, " completed!")

100%|███████████████████████████████████████████████████| 289992/289992 [00:02<00:00, 140800.96it/s]


all_list2/体育健身.xlsx  completed!


100%|████████████████████████████████████████████████████| 289992/289992 [00:13<00:00, 21480.75it/s]


all_list2/交通设施.xlsx  completed!


100%|███████████████████████████████████████████████████| 289992/289992 [00:01<00:00, 186582.08it/s]


all_list2/文化展览.xlsx  completed!


100%|████████████████████████████████████████████████████| 289992/289992 [00:17<00:00, 16788.04it/s]


In [63]:
def places_len(places_np):
    max_len = 1
    min_len = 10
    for p in places_np:
        max_len = max(len(p), max_len)   
        min_len = min(len(p), min_len)   
    return (max_len, min_len)

def min_places(places_np, min_lenght = 1):
    min_p = []
    for p in places_np:
        if len(p) == min_lenght:
            min_p.append(p)
            
    return min_p

In [64]:
# save match results with pickle
headers = ["相关地点数量", "微博id", "微博主页", "文本", "日期", "图片", "转发数", "评论数", "点赞数"]
names2loc_key = set(list(names2loc.keys()))

for lst in ['list2_img']:
    
    text_csv = 'all_list/' + '%s.csv'%(lst)
    text_pickle = 'all_list2/' + '*_%s_*.pickle'%(lst)
    text_pickles = glob.glob(text_pickle)
    this_texts = pd.read_csv(text_csv, index_col=0)
    
    for t in text_pickles:
        append_cells = {}
        with open(t, 'rb') as f:
            [places, sparse_matirx] = pickle.load(f)
        places_np = np.array(places)
        sparse_np = np.array(sparse_matirx) #.shape
        if t.split('/')[-1][1].isdigit():
            title_name = sub_names[int(t.split('/')[-1][:2])-1]
        else:
            title_name = sub_names[int(t.split('/')[-1][0])-1]
        
        print("{:>12s}".format(title_name), " max: {:2>d} min: {:2>d}".format(places_len(places_np)[0], places_len(places_np)[1]), end= ' ')
        min_ps = min_places(places_np)
        if len(min_ps):
            print(min_ps, end=' ')
        print('')

        体育健身  max: 13 min: 2 
        交通设施  max: 19 min: 1 ['内'] 
        文化展览  max: 13 min: 2 
        商业娱乐  max: 22 min: 1 ['顾', '尚', 'R', '简', '锦', '落', '嚟', 'C', 'e', 'f', 'H', 'L', 'n', 'W', '酷', '漫', '品', '陶', '喜', '懿', '银', '渔', '遇', '青'] 
        企业办公  max: 17 min: 2 
        公共服务  max: 8 min: 2 
        旅游景点  max: 16 min: 1 ['佛'] 
        医养设施  max: 16 min: 1 ['中'] 
        科研教育  max: 19 min: 2 
          住区  max: 14 min: 2 


In [67]:
full_names = pd.read_excel('/home/yangwenhao/local/project/weibo_filter/app/data/POI_tianjin_0427.xlsx',
                              sheet_name=0)

jingdu = full_names.经度 
weidu = full_names.纬度
print(type(jingdu.to_list()[0]))

In [73]:
jingwei = []
for x,y in zip(jingdu.to_list(), weidu.to_list()):
    jingwei.append(str(x) + '_' + str(y))

In [74]:
uniq_jingwei = set(jingwei)

In [75]:
print(len(jingwei), len(uniq_jingwei))

389737 389737


In [77]:
jingwei.sort()
jingwei[:6]

['116.709156_38.910404',
 '116.709434_38.910285',
 '116.709707_38.910545',
 '116.71021_38.91017',
 '116.711383_38.91127',
 '116.71213_38.912131']